# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Kaggle Assignment: **

**Student Name: Jason Walker**

# Overview

## Description
This is one of the projects from the course T81-855: Applications of Deep Learning at Washington University in St. Louis. All students must create a Kaggle account and submit a solution. Once you have submitted your solution entry log into Blackboard (at WUSTL) and submit a single file telling me your Kaggle name on the leaderboard (you do not need to register to Kaggle with your real name). This competition will be visible to the public, so there may be non-student submissions as well as student.

The data set for this competition consists of a number of input columns that should be used to predict a stores sales. This is a regression problem. The inputs are a mixture of discrete and category values. The data set is from a simulation.

## Evaluation

The evaluation pages describes how submissions will be scored and how students should format their submissions. The scores are in RMSE.
Submission Format

For every store in the dataset, submission files should contain a sales volume.

The file should contain a header and have the following format:

```
100000,1.23
100001,1.123
100002,3.332
100003,1.53
...
```

# Data

## Data Description

The data contains data and costs for various office supplies. The data came from a simulation and do not directly correspond to any real-world items. See how well you can predict the cost of an item using the provided data. Feature engineering will likely help you. The *name* column may seem useless at first glance; however, it contains information that you can parse to help your predictions.
File descriptions
```
    id - The identifier/primary key.
    name - The name of this item.
    manufacturer - The manufacturer.
    pack - The number of items in this pack.
    weight - The weight of a pack of these items.
    height - The height of a pack of these items.
    width - The width of a pack of these items.
    length - The length of a pack of these items.
    cost - The cost for this item pack. This is what you are to predict (the target). 
```

# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Kaggle Code

## Load Data and Encode


In [2]:
import os
import re
import sklearn

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

path = './data'

filename_test = os.path.join(path,"test.csv")
filename_train = os.path.join(path,"train.csv")
filename_sample = os.path.join(path,"sample.csv")
filename_submit = os.path.join(path,"submit.csv")
filename_checkpoint = os.path.join(path,"checkpoint.hdf5")

df_train = pd.read_csv(filename_train,na_values=['NA','?'])

np.random.seed(42) # Uncomment this line to get the same shuffle each time
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.reset_index(inplace=True, drop=True)

# Encode Features

# Remove all rows where the specified column is +/- sd standard deviations
def extract_and_encode_features(df):
    color_regex='(?P<color>red|blue|green|yellow|orange|pink|black|brown|white)'
    df['color'] = df.name.str.extract(color_regex, flags=re.IGNORECASE, expand=False)

    quality_regex='(?P<quality>generic|high\squality)'
    df['quality'] = df.name.str.extract(quality_regex, flags=re.IGNORECASE, expand=False)

    size_regex='(?P<size>tiny|small|medium|large)'
    df['size'] = df.name.str.extract(size_regex, flags=re.IGNORECASE, expand=False)

    item_regex='(?P<item>paperclips|paperweights|ink\spens|pencils|stapler|tablets|thumbtacks|post\sit\snotes)'
    df['item'] = df.name.str.extract(item_regex, flags=re.IGNORECASE, expand=False)
    
    for column in ['pack','weight','height','width','length']:
        missing_median(df,column)
    
    #df.insert(1,'surface_area',(df['height']*df['width']*df['length']).astype(int))
    
    ## encode numeric features
    #for column in ['pack','weight','height','width','length','surface_area']:
    #    encode_numeric_zscore(df,column)

    # encode text/categorical features
    for column in ['manufacturer','color','quality','size','item']:
        encode_text_dummy(df,column)
  
extract_and_encode_features(df_train)

ids_train = df_train['id']
df_train.drop('id',1,inplace=True)

names_train = df_train['name']
df_train.drop('name',1,inplace=True)

x,y = to_xy(df_train,'cost')

# Used before KFold
#x_train, x_test, y_train, y_test = train_test_split(    
#    x, y, test_size=0.25, random_state=45)


/Users/jwalker/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


## Linear Regression (Coefficients)

In [ ]:
# Simple function to evaluate the coefficients of a regression
%matplotlib inline    
from IPython.display import display, HTML    

def report_coef(names,coef,intercept):
    r = pd.DataFrame( { 'coef': coef, 'positive': coef>=0  }, index = names )
    r = r.sort_values(by=['coef'])
    display(r)
    print("Intercept: {}".format(intercept))
    r['coef'].plot(kind='barh', color=r['positive'].map({True: 'b', False: 'r'}))
    
# Create linear regression
regressor = sklearn.linear_model.LinearRegression()

# Fit/train linear regression
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")
print(names)
report_coef(
  names,
  regressor.coef_[0,:],
  regressor.intercept_)

## L1 Regularization (Lasso)

In [ ]:


import sklearn
from sklearn.linear_model import Lasso

# Create linear regression
regressor = Lasso(random_state=0,alpha=0.01)

# Fit/train LASSO
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")
report_coef(
  names,
  regressor.coef_,
  regressor.intercept_)



## L1 Regularization (Plot LassoCV)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

lasso = Lasso(random_state=42)
alphas = np.logspace(-8, 8, 10)

scores = list()
scores_std = list()

n_folds = 3

for alpha in alphas:
    lasso.alpha = alpha
    this_scores = cross_val_score(lasso, x, y, cv=n_folds, n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

scores, scores_std = np.array(scores), np.array(scores_std)

plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores)

# plot error lines showing +/- std. errors of the scores
std_error = scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, scores + std_error, 'b--')
plt.semilogx(alphas, scores - std_error, 'b--')

# alpha=0.2 controls the translucency of the fill color
plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

plt.ylabel('CV score +/- std error')
plt.xlabel('alpha')
plt.axhline(np.max(scores), linestyle='--', color='.5')
plt.xlim([alphas[0], alphas[-1]])

## L2 Regularization (Ridge)

In [ ]:
import sklearn
from sklearn.linear_model import Ridge

# Create linear regression
regressor = Ridge(alpha=1)

# Fit/train Ridge
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")

report_coef(
  names,
  regressor.coef_[0,:],
  regressor.intercept_)

## ElasticNet Regularization

In [ ]:
import sklearn
from sklearn.linear_model import ElasticNet

# Create linear regression
regressor = ElasticNet(alpha=0.01, l1_ratio=0.1)

# Fit/train LASSO
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")

report_coef(
  names,
  regressor.coef_,
  regressor.intercept_)

## Generate Training Model

In [3]:
# Cross-Validate
kf = KFold(5)

checkpoint = ModelCheckpoint(filepath=filename_checkpoint, verbose=0, save_best_only=True)

# Turn off KFold
#if (0):
oos_y = []
oos_pred = []
fold = 0
    
for train, test in kf.split(x):
    fold+=1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dropout(0.01)) # Dropout Layer 1
    model.add(Dense(50, activation='relu')) # Hidden 2
    #model.add(Dropout(0.01)) # Dropout Layer 2
    model.add(Dense(25, 
                    kernel_regularizer=regularizers.l2(0.01), #L2 regularization
                    activity_regularizer=regularizers.l1(0.01), #L1 Lasso regularization
                    activation='relu')) # Hidden 3 
    model.add(Dense(10, activation='relu')) # Hidden 4
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=1000, verbose=1, mode='auto')
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpoint],verbose=1,epochs=10000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)        

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))


Fold #1
Train on 8000 samples, validate on 2000 samples
Epoch 1/10000
8000/8000 [==============================] - 1s 120us/step - loss: 9890.1824 - val_loss: 5246.3600
Epoch 2/10000
8000/8000 [==============================] - 0s 54us/step - loss: 5819.8733 - val_loss: 4713.2834
Epoch 3/10000
8000/8000 [==============================] - 0s 56us/step - loss: 5355.4098 - val_loss: 4503.2949
Epoch 4/10000
8000/8000 [==============================] - 0s 53us/step - loss: 4955.7231 - val_loss: 4227.3757
Epoch 5/10000
8000/8000 [==============================] - 0s 53us/step - loss: 4686.3512 - val_loss: 3867.3678
Epoch 6/10000
8000/8000 [==============================] - 0s 54us/step - loss: 4450.1424 - val_loss: 3778.2384
Epoch 7/10000
8000/8000 [==============================] - 0s 53us/step - loss: 4349.5404 - val_loss: 3743.2444
Epoch 8/10000
8000/8000 [==============================] - 0s 51us/step - loss: 4184.5648 - val_loss: 5050.9867
Epoch 9/10000
8000/8000 [======================

Epoch 74/10000
8000/8000 [==============================] - 0s 48us/step - loss: 370.0371 - val_loss: 258.3468
Epoch 75/10000
8000/8000 [==============================] - 0s 49us/step - loss: 324.4351 - val_loss: 435.5742
Epoch 76/10000
8000/8000 [==============================] - 0s 56us/step - loss: 382.2652 - val_loss: 380.6266
Epoch 77/10000
8000/8000 [==============================] - 0s 47us/step - loss: 492.0190 - val_loss: 338.2193
Epoch 78/10000
8000/8000 [==============================] - 0s 52us/step - loss: 454.8989 - val_loss: 754.7912
Epoch 79/10000
8000/8000 [==============================] - 0s 46us/step - loss: 337.9105 - val_loss: 335.8611
Epoch 80/10000
8000/8000 [==============================] - 0s 45us/step - loss: 333.0839 - val_loss: 350.6711
Epoch 81/10000
8000/8000 [==============================] - 0s 47us/step - loss: 375.0729 - val_loss: 472.4104
Epoch 82/10000
8000/8000 [==============================] - 0s 45us/step - loss: 382.3224 - val_loss: 416.9804
E

8000/8000 [==============================] - 0s 48us/step - loss: 258.9568 - val_loss: 308.6782
Epoch 148/10000
8000/8000 [==============================] - 0s 47us/step - loss: 293.5676 - val_loss: 274.8468
Epoch 149/10000
8000/8000 [==============================] - 0s 46us/step - loss: 261.3405 - val_loss: 212.4020
Epoch 150/10000
8000/8000 [==============================] - 0s 48us/step - loss: 266.0192 - val_loss: 284.1592
Epoch 151/10000
8000/8000 [==============================] - 0s 46us/step - loss: 265.1651 - val_loss: 202.2034
Epoch 152/10000
8000/8000 [==============================] - 0s 46us/step - loss: 270.6080 - val_loss: 266.0550
Epoch 153/10000
8000/8000 [==============================] - 0s 46us/step - loss: 284.2711 - val_loss: 261.3678
Epoch 154/10000
8000/8000 [==============================] - 0s 46us/step - loss: 320.9084 - val_loss: 228.8696
Epoch 155/10000
8000/8000 [==============================] - 0s 46us/step - loss: 253.3033 - val_loss: 201.9169
Epoch 15

8000/8000 [==============================] - 0s 47us/step - loss: 390.9184 - val_loss: 271.7801
Epoch 221/10000
8000/8000 [==============================] - 0s 47us/step - loss: 330.6633 - val_loss: 226.0698
Epoch 222/10000
8000/8000 [==============================] - 0s 45us/step - loss: 305.7132 - val_loss: 203.3361
Epoch 223/10000
8000/8000 [==============================] - 0s 47us/step - loss: 247.2867 - val_loss: 232.3582
Epoch 224/10000
8000/8000 [==============================] - 0s 46us/step - loss: 312.1656 - val_loss: 198.8888
Epoch 225/10000
8000/8000 [==============================] - 0s 46us/step - loss: 241.5766 - val_loss: 237.4470
Epoch 226/10000
8000/8000 [==============================] - 0s 48us/step - loss: 234.7928 - val_loss: 296.8653
Epoch 227/10000
8000/8000 [==============================] - 0s 47us/step - loss: 245.7456 - val_loss: 250.4706
Epoch 228/10000
8000/8000 [==============================] - 0s 46us/step - loss: 306.6969 - val_loss: 226.2948
Epoch 22

8000/8000 [==============================] - 0s 47us/step - loss: 189.8544 - val_loss: 173.2056
Epoch 294/10000
8000/8000 [==============================] - 0s 46us/step - loss: 255.9976 - val_loss: 189.5207
Epoch 295/10000
8000/8000 [==============================] - 0s 46us/step - loss: 205.6169 - val_loss: 231.5832
Epoch 296/10000
8000/8000 [==============================] - 0s 47us/step - loss: 234.4024 - val_loss: 232.4367
Epoch 297/10000
8000/8000 [==============================] - 0s 45us/step - loss: 244.9484 - val_loss: 248.5802
Epoch 298/10000
8000/8000 [==============================] - 0s 46us/step - loss: 212.7049 - val_loss: 246.0919
Epoch 299/10000
8000/8000 [==============================] - 0s 44us/step - loss: 343.3253 - val_loss: 215.4778
Epoch 300/10000
8000/8000 [==============================] - 0s 45us/step - loss: 214.8449 - val_loss: 182.8334
Epoch 301/10000
8000/8000 [==============================] - 0s 44us/step - loss: 214.0561 - val_loss: 178.9881
Epoch 30

...

8000/8000 [==============================] - 0s 62us/step - loss: 240.8096 - val_loss: 151.3730
Epoch 347/10000
8000/8000 [==============================] - 1s 68us/step - loss: 248.8822 - val_loss: 245.5599
Epoch 348/10000
8000/8000 [==============================] - 1s 69us/step - loss: 193.5892 - val_loss: 143.3253
Epoch 349/10000
8000/8000 [==============================] - 0s 59us/step - loss: 206.5140 - val_loss: 151.2720
Epoch 350/10000
8000/8000 [==============================] - 0s 52us/step - loss: 210.2462 - val_loss: 147.7201
Epoch 351/10000
8000/8000 [==============================] - 0s 51us/step - loss: 213.0258 - val_loss: 186.7118
Epoch 352/10000
8000/8000 [==============================] - 0s 52us/step - loss: 195.1841 - val_loss: 151.5850
Epoch 353/10000
8000/8000 [==============================] - 0s 55us/step - loss: 219.8472 - val_loss: 138.9260
Epoch 354/10000
8000/8000 [==============================] - 0s 51us/step - loss: 179.6433 - val_loss: 147.5795
Epoch 35

8000/8000 [==============================] - 1s 69us/step - loss: 225.6963 - val_loss: 168.9105
Epoch 420/10000
8000/8000 [==============================] - 1s 66us/step - loss: 193.2495 - val_loss: 207.1532
Epoch 421/10000
8000/8000 [==============================] - 0s 55us/step - loss: 200.9445 - val_loss: 135.6189
Epoch 422/10000
8000/8000 [==============================] - 0s 52us/step - loss: 218.7209 - val_loss: 144.9700
Epoch 423/10000
8000/8000 [==============================] - 0s 52us/step - loss: 199.8270 - val_loss: 243.7440
Epoch 424/10000
8000/8000 [==============================] - 0s 53us/step - loss: 187.4639 - val_loss: 155.6383
Epoch 425/10000
8000/8000 [==============================] - 0s 51us/step - loss: 226.7575 - val_loss: 340.0275
Epoch 426/10000
8000/8000 [==============================] - 0s 50us/step - loss: 199.7939 - val_loss: 172.4631
Epoch 427/10000
8000/8000 [==============================] - 0s 52us/step - loss: 198.3277 - val_loss: 147.2791
Epoch 42

8000/8000 [==============================] - 0s 47us/step - loss: 219.8327 - val_loss: 140.1226
Epoch 493/10000
8000/8000 [==============================] - 0s 47us/step - loss: 187.7571 - val_loss: 136.2064
Epoch 494/10000
8000/8000 [==============================] - 0s 48us/step - loss: 194.1597 - val_loss: 173.7092
Epoch 495/10000
8000/8000 [==============================] - 0s 45us/step - loss: 202.1550 - val_loss: 141.6891
Epoch 496/10000
8000/8000 [==============================] - 0s 46us/step - loss: 187.2245 - val_loss: 129.2815
Epoch 497/10000
8000/8000 [==============================] - 0s 46us/step - loss: 191.9860 - val_loss: 130.0535
Epoch 498/10000
8000/8000 [==============================] - 0s 46us/step - loss: 196.6177 - val_loss: 141.6924
Epoch 499/10000
8000/8000 [==============================] - 0s 46us/step - loss: 241.1085 - val_loss: 137.7501
Epoch 500/10000
8000/8000 [==============================] - 0s 47us/step - loss: 188.5755 - val_loss: 144.1121
Epoch 50

8000/8000 [==============================] - 0s 62us/step - loss: 211.6176 - val_loss: 160.5619
Epoch 566/10000
8000/8000 [==============================] - 0s 48us/step - loss: 250.8884 - val_loss: 168.0935
Epoch 567/10000
8000/8000 [==============================] - 0s 46us/step - loss: 212.3380 - val_loss: 135.0296
Epoch 568/10000
8000/8000 [==============================] - 0s 54us/step - loss: 202.2939 - val_loss: 127.2839
Epoch 569/10000
8000/8000 [==============================] - 1s 64us/step - loss: 189.0346 - val_loss: 145.2117
Epoch 570/10000
8000/8000 [==============================] - 0s 48us/step - loss: 175.2529 - val_loss: 124.5822
Epoch 571/10000
8000/8000 [==============================] - 0s 49us/step - loss: 180.5656 - val_loss: 150.9253
Epoch 572/10000
8000/8000 [==============================] - 0s 49us/step - loss: 177.0951 - val_loss: 137.9971
Epoch 573/10000
8000/8000 [==============================] - 0s 52us/step - loss: 183.6173 - val_loss: 192.2125
Epoch 57

8000/8000 [==============================] - 0s 53us/step - loss: 186.2918 - val_loss: 151.8640
Epoch 639/10000
8000/8000 [==============================] - 0s 52us/step - loss: 184.1792 - val_loss: 159.6387
Epoch 640/10000
8000/8000 [==============================] - 0s 50us/step - loss: 181.3420 - val_loss: 199.2456
Epoch 641/10000
8000/8000 [==============================] - 0s 48us/step - loss: 183.8585 - val_loss: 133.4702- ETA: 0s - loss: 184.
Epoch 642/10000
8000/8000 [==============================] - 0s 46us/step - loss: 170.9659 - val_loss: 217.6070
Epoch 643/10000
8000/8000 [==============================] - 0s 46us/step - loss: 200.8481 - val_loss: 138.0464
Epoch 644/10000
8000/8000 [==============================] - 0s 60us/step - loss: 247.8519 - val_loss: 137.5231
Epoch 645/10000
8000/8000 [==============================] - 0s 59us/step - loss: 220.8420 - val_loss: 132.2742
Epoch 646/10000
8000/8000 [==============================] - 0s 55us/step - loss: 204.1068 - val_l

## Out-of-sample RMSE

In [4]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print("Final, out of sample score (RMSE): {}".format(score)) 

Final, out of sample score (RMSE): 12.610228538513184


In [ ]:

#pred = model.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
#score = np.sqrt(metrics.mean_squared_error(pred,y_test))
#print("Final score (RMSE): {}".format(score))

#chart_regression(pred.flatten(),y_test)
#chart_regression(pred.flatten(),y_test,sort=False)

## Load Best Weights and Predict on Test

In [5]:
model.load_weights(filename_checkpoint)

df_test = pd.read_csv(filename_test,na_values=['NA','?'])

extract_and_encode_features(df_test)

ids_test = df_test['id']
df_test.drop('id',1,inplace=True)

names_test = df_test['name']
df_test.drop('name',1,inplace=True)

x_submit = df_test.as_matrix().astype(np.float32)
pred_submit = model.predict(x_submit)

# Handles negative cost values. Use inverse
cost = [n if n > 0 else n * -1 for n in pred_submit[:,0]]

df_submit = pd.DataFrame({'id': ids_test,'cost': cost})
df_submit = df_submit[['id', 'cost']]
df_submit.to_csv(filename_submit, index=False)